# Chatbot with Conversational History using LCEL

In [3]:
import os 
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [4]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "gemma2-9b-It", groq_api_key = groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001DE269C2540>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001DE26B04E90>, model_name='gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content = "Hii, I am Yash. I am a Data Sceintist")])

AIMessage(content="Hello Yash! 👋\n\nIt's great to meet you. Being a Data Scientist is awesome! What kind of projects are you working on these days?  \n\nDo you have any questions for me about data science, or anything else? I'm here to help! 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 22, 'total_tokens': 83, 'completion_time': 0.110909091, 'prompt_time': 0.00015399, 'queue_time': 0.013471419, 'total_time': 0.111063081}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e85f87c6-a613-4024-8ab9-368c4613e08b-0', usage_metadata={'input_tokens': 22, 'output_tokens': 61, 'total_tokens': 83})

In [6]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content = "Hii, I am Yash. I am a Data Sceintist"),
        AIMessage(content = "Hello Yash! 👋\n\nIt's great to meet you. Being a Data Scientist is awesome! What kind of projects are you working on these days?  \n\nDo you have any questions for me about data science, or anything else? I'm here to help! 😊 \n\n"),
        HumanMessage(content = "What is my name and what do I do?"),
    ]
)

AIMessage(content="You told me your name is Yash, and that you are a Data Scientist!  \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm curious to learn more. 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 100, 'total_tokens': 149, 'completion_time': 0.089090909, 'prompt_time': 0.002506402, 'queue_time': 0.012826656, 'total_time': 0.091597311}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-23b259d7-243a-49f6-8ad2-5bc31a3977d9-0', usage_metadata={'input_tokens': 100, 'output_tokens': 49, 'total_tokens': 149})

It seems that our model is remembering out history.

### **Associating Conversation History with SessionId**

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

config = {
    "configurable":{
        "session_id": "chat1"
    }
}

In [9]:
response = with_message_history.invoke(
    [HumanMessage(content= "Hi my name is Yash, I am a Data Scientist")],
    config = config
)
response.content

"It's nice to meet you, Yash!  \n\nWhat kind of data science work do you do?  Are you working on any interesting projects right now? 😊 \n\nI'm always eager to learn more about how people are using data science to make a difference.  \n\n"

In [10]:
with_message_history.invoke(
    [
        HumanMessage(content= "What is my name")
    ],
    config = config
)

AIMessage(content='Your name is Yash. 😊  \n\nI remember that you told me at the beginning of our conversation.  \n\n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 173, 'total_tokens': 199, 'completion_time': 0.047272727, 'prompt_time': 0.005363543, 'queue_time': 0.008265204, 'total_time': 0.05263627}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f330d6ef-e88d-4ba5-b3d6-9b3b5fc63b12-0', usage_metadata={'input_tokens': 173, 'output_tokens': 26, 'total_tokens': 199})

In [ ]:
config2 = {
    "configurable":{
        "session_id": "chat2"
    }
}
}